In [ ]:
# ADVANCED RAG INTEGRATION
from ollama import chat
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import os
import re

DOCUMENT_PATHS = [
    r'C:\Users\ASUS\Downloads\Hamrah.txt', #replace path
    r'C:\Users\ASUS\Downloads\vape.txt',
    r'C:\Users\ASUS\Downloads\Shah.txt',
    r'C:\Users\ASUS\Downloads\Khalife.txt',
    r'C:\Users\ASUS\Downloads\carbon.txt',
    r'C:\Users\ASUS\Downloads\takapoo.txt',
    r'C:\Users\ASUS\Downloads\mahmood.txt'
]

EMBEDDING_MODEL = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
LLM_MODEL = 'llama3.2'
CHUNK_SIZE = 1000
OVERLAP = 200
INDEX_PATH = r'C:\Users\ASUS\Downloads\doc_index.faiss'
CHUNK_MAP_PATH = r'C:\Users\ASUS\Downloads\chunk_map.npy'

class AdvancedRAG:
    def __init__(self):
        self.encoder = SentenceTransformer(EMBEDDING_MODEL)
        self.index = None
        self.chunk_map = []
        
    def create_index(self):
        """Create FAISS index with cosine similarity and document mapping"""
        all_chunks = []
        doc_mapping = []
        
        # Process via CHUNKING (REQ 4 RAG)
        for doc_idx, path in enumerate(DOCUMENT_PATHS):
            with open(path, 'r', encoding='utf-8') as f:
                text = re.sub(r'\s+', ' ', f.read()).strip()
                chunks = [text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE - OVERLAP)]
                all_chunks.extend(chunks)
                doc_mapping.extend([doc_idx] * len(chunks))
        
        # Normalized embeddings (REQ 4 cosine similarity)
        embeddings = self.encoder.encode(all_chunks)
        faiss.normalize_L2(embeddings)  
        
        # FAISS index & Mapping
        self.index = faiss.IndexFlatIP(embeddings.shape[1])
        self.index.add(embeddings.astype(np.float32))
        self.chunk_map = np.array(doc_mapping)
        
        # Index  
        faiss.write_index(self.index, INDEX_PATH)
        # Mapping 
        np.save(CHUNK_MAP_PATH, self.chunk_map)
    
    def load_index(self):
        """LOAD EXISTING DATA"""
        self.index = faiss.read_index(INDEX_PATH)
        self.chunk_map = np.load(CHUNK_MAP_PATH, allow_pickle=True)
        
    def query(self, question, doc_index, top_k=6):
        """DOCUMENT-SPECIFIC QUERY WITH COSINE SIMILARITY """
        # Encode 
        query_embed = self.encoder.encode([question])
        # Normalize 
        faiss.normalize_L2(query_embed)
        
        distances, indices = self.index.search(query_embed.astype(np.float32), top_k*3)
    
        relevant_chunks = []
        for idx in indices[0]:
            if self.chunk_map[idx] == doc_index:
                relevant_chunks.append(idx)
            if len(relevant_chunks) >= top_k:
                break
        
        return relevant_chunks

class AnswerGenerator:
    def __init__(self, rag_system):
        self.rag = rag_system
        self.chunks = [] 
        
    def get_answer(self, question, doc_index):
        """GENERATING CONTEXT-AWARE ANSWER"""
        if not self.chunks:
            self._load_chunks()
            
        chunk_indices = self.rag.query(question, doc_index)
        context = "\n".join([self.chunks[idx] for idx in chunk_indices])
        
        prompt = f"""با استفاده از متن زیر به سوال پاسخ دهید:
{context}

اگر پاسخ در متن وجود ندارد عبارت 'پاسخی یافت نشد' را برگردانید

سوال: {question}
پاسخ:"""
        
        response = chat(model=LLM_MODEL, messages=[{'role': 'user', 'content': prompt}])
        return response['message']['content']
    
    def _load_chunks(self):
        """LOAD ALL CHUNKS(LAZY)"""
        self.chunks = []
        for path in DOCUMENT_PATHS:
            with open(path, 'r', encoding='utf-8') as f:
                text = re.sub(r'\s+', ' ', f.read()).strip()
                self.chunks.extend([text[i:i+CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE - OVERLAP)])

# MAIN EXE of RAG
if __name__ == "__main__":
    # RAG init
    rag = AdvancedRAG()
    
    if not os.path.exists(INDEX_PATH):
        print("Building optimized index...")
        rag.create_index()
    else:
        print("Loading existing index...")
        rag.load_index()
    # Answer Generator init
    generator = AnswerGenerator(rag)
    
    queries = [
        ("چرا اینترنت همراه اول گوشی وصل نمیشود؟", 0),
        ("چطوری ویپ مورد نظرمو پیدا کنم؟", 1),
        ("شاه عباس که بود؟", 2),
        ("خلیفه سلطان که بود و چه کرد؟", 3),
        ("کربن اکتیو و کربن بلک چه هستند و چه تفاوتی دارند و برای چه استفاده میشن؟", 4),
        ("شرکت تکاپو صنعت نامی چه محصولاتی ارایه میدهد؟ چه چیزی این شرکت را منحصر به فرد میسازد؟ سهام این شرکت صعودی است یا نزولی؟", 5),
        ("6 ,"سید محمود خلیفه سلطانی کیست؟"),
    ]
    
    with open(r'C:\Users\ASUS\Downloads\representation.txt', 'w', encoding='utf-8') as f: #replace path
        for q_idx, (query, doc_idx) in enumerate(queries):
            answer = generator.get_answer(query, doc_idx)
            f.write(f"سوال {q_idx+1} ({doc_idx+1}):\n{query}\n\nپاسخ:\n{answer}\n\n{'='*50}\n\n")
            print(f"پردازش سوال {q_idx+1}/{len(queries)} تکمیل شد")

print("تمامی سوالات با موفقیت پردازش شدند!")